#Hedging Stock Portfolio with Futures

###**Imports & Data Acquisition**

####Imports

In [ ]:
import datetime as dt

import numpy as np
import pandas as pd
import yfinance as yf

import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.dates as mdates
import seaborn as sns

from sklearn.linear_model import LinearRegression

import quantstats as qs


####Download Stock Data

In [ ]:
def return_stocks(stocks, end_date=dt.datetime.now(), years=5):
    """
    Retrieves historical stock data for a list of stock tickers and calculates their daily logarithmic returns.

    This function downloads historical stock data for the specified tickers over a given period, calculates
    the daily logarithmic returns for each stock, and returns two pandas DataFrames: one containing the log returns and
    another containing the closing prices. The function also handles missing data by performing linear interpolation.

    Parameters:
    stocks (list or str): A list of stock tickers (as strings) or a single stock ticker.
    end_date (datetime): The end date for the historical data. Defaults to the current date.
    years (int): The number of years of historical data to retrieve. Defaults to 5 years.

    Returns:
    tuple: A tuple containing:
        - pandas.DataFrame: The daily log returns of the stocks.
        - pandas.DataFrame: The historical closing prices of the stocks.
    """

    start_date = end_date - dt.timedelta(days=366 * years)
    historical = yf.download(stocks, start=start_date, end=end_date)

    # Extract the closing prices from the historical data
    historical = historical['Close']

    # Ensure the historical data is sorted by date in ascending order
    historical = historical.sort_index()

    # Perform linear interpolation to fill in any missing values in the historical closing prices
    historical = historical.interpolate(method='linear', axis=0)

    return historical


####Loading and Cleaning FTSE ATHEX Large Cap Price Data

In [ ]:
# Import FTSE data and clean it

# Read FTSE price data from a CSV file
ftse_price = pd.read_csv('/content/FTSE_ATHEX Large Cap Historical Data.csv')

# Extract the 'Date' column to use as index
i = ftse_price['Date']

# Extract 'Price' column as the closing price
ftse_price = ftse_price['Price']

# Set the 'Date' column as the index
ftse_price.index = i

# Convert the price data into a DataFrame
ftse_price = pd.DataFrame(ftse_price)

# Rename the column to 'Close' for consistency with other datasets
ftse_price.columns = ['Close']

# Rename the index axis to 'Date'
ftse_price = ftse_price.rename_axis('Date')

# Reset the index to make 'Date' a column again
ftse_price = ftse_price.reset_index()

# Convert 'Date' column to datetime format (month/day/year)
ftse_price['Date'] = pd.to_datetime(ftse_price['Date'], format='%m/%d/%Y')

# Function to replace commas with dots in the price data (for proper float conversion)
def swap_dots_commas(value):
    return value.replace(',', '')

# Apply the function to the 'Close' column to clean the data
ftse_price['Close'] = ftse_price['Close'].apply(swap_dots_commas)

# Convert the 'Close' column from string to float
ftse_price['Close'] = ftse_price['Close'].astype(float)

# Set the 'Date' column as the index again after cleaning
ftse_price.set_index('Date', inplace=True)

# Sort the data by date in ascending order
ftse_price = ftse_price.sort_index()


####Inputs

In [ ]:
# Define stock symbols for the portfolio
stocks = ["TENERGY.AT", "HTO.AT", "KRI.AT", "PPC.AT", "EUROB.AT", "INKAT.AT",
          "INLOT.AT", "PREMIA.AT", "LAVI.AT", "LAMDA.AT", "OTOEL.AT", "OPAP.AT"]

# Define portfolio weights corresponding to each stock
port_weights = [0.110147, 0.094247, 0.077479, 0.090489, 0.090199, 0.089621,
                0.078346, 0.069673, 0.038450, 0.093958, 0.087019, 0.080370]

# Retrieve historical stock prices for the defined stocks over the last 2 years up to the specified end date
stock_prices = return_stocks(stocks, end_date=dt.datetime(2024, 8, 10), years=2)



[*********************100%***********************]  12 of 12 completed


In [ ]:
# INPUTS FOR OUR FUNCTION

# Portfolio weights defined earlier, representing the allocation for each stock
port_weights = port_weights

# Index prices for the FTSE, used as a benchmark or reference for performance
index_prices = ftse_price

# Total equity amount available for investment
equity = 100000

# Frequency of rebalancing the portfolio (in months)
frequency = 3

# Required margin for trading (10% of the equity)
req_margin = 0.1

# Index multiplier, which determines the overall contract price for index futures
multiplier = 2

# Start date for the investment period
start_date = dt.datetime(2023, 8, 10)

# End date for the investment period, initialized as 0 for future assignment
end_date = 0